In [1]:
import shapely.geometry
import time, requests
import json

from IPython.display import display
from ipywidgets import *
from ipyleaflet import *
from ipydatetime import DatetimePicker
from osprey_flask_app import create_app

In [2]:
f = open("domains.json")
data = json.load(f)

peace_coords = [
    (coord[0], coord[1]) for coord in data["borders"]["peace"]["coordinates"]
]
fraser_coords = [
    (coord[0], coord[1]) for coord in data["borders"]["fraser"]["coordinates"]
]
columbia_coords = [
    (coord[0], coord[1]) for coord in data["borders"]["columbia"]["coordinates"]
]

In [3]:
def handle_click(**kwargs):
    if kwargs.get("type") == "click":
        lat = round(kwargs.get("coordinates")[0], 5)
        lon = round(kwargs.get("coordinates")[1], 5)

        if in_polygon(lat, lon):
            clear_previous_marker()

            latitude.value = f"{lat}"
            longitude.value = f"{lon}"

            m.add_layer(Marker(location=kwargs.get("coordinates"), name="Marker"))


def handle_button(arg):
    print("The button has been clicked")

def handle_add(arg):
    
    if f"{latitude.value}, {longitude.value}" not in points:
        points.append(f"{latitude.value}, {longitude.value}")
        point_list.options = points
    
    
def handle_remove(arg):
    
    if point_list.value:
        for point in point_list.value:
            points.remove(point)
            
    point_list.options = points

In [4]:
def clear_previous_marker():
    for i in range(1, len(m.layers)):
        if m.layers[i].name == "Marker":
            m.remove_layer(m.layers[i])


def in_polygon(lat, lon):

    point = shapely.geometry.Point(lat, lon)

    peace_polygon = shapely.geometry.polygon.Polygon(p.locations)
    fraser_polygon = shapely.geometry.polygon.Polygon(f.locations)
    columbia_polygon = shapely.geometry.polygon.Polygon(c.locations)

    if (
        peace_polygon.contains(point)
        or fraser_polygon.contains(point)
        or columbia_polygon.contains(point)
    ):
        return True

    return False


def date_widget(descr):
    return DatetimePicker(description=descr, disabled=False)

In [5]:
mapnik = basemap_to_tiles(basemaps.OpenStreetMap.Mapnik)
mapnik.base = True
mapnik.name = "Default"

satellite = basemap_to_tiles(basemaps.Gaode.Satellite)
satellite.base = True
satellite.name = "Satellite"

m = Map(
    basemap=mapnik,
    center=(50.5, -120),
    zoom=5,
    layout=Layout(width="50%", height="800px"),
    layers=[satellite, mapnik],
)

p = Polygon(
    locations=peace_coords,
    color="blue",
    name="Peace",
)
f = Polygon(
    locations=fraser_coords,
    color="red",
    name="Fraser",
)
c = Polygon(
    locations=columbia_coords,
    color="green",
    name="Columbia",
)

legend = LegendControl(
    {"Peace": "blue", "Fraser": "red", "Columbia": "green"},
    name="Watersheds",
    position="topright",
)

latitude = Text(
    placeholder="Enter the Latitude", description="Latitude:", disabled=False
)

longitude = Text(
    placeholder="Enter the Longitude", description="Longitude:", disabled=False
)

layer_control = LayersControl()

point_list = SelectMultiple(options=[], value=[], description="Points:", disabled=False)

points = []

run = Button(
    description="Run",
    button_style="success",
    disabled=False,
    tooltip="Click 'Run' to start the Osprey Flask App.",
)
run.on_click(handle_button)

add_point = Button(
    description="Add Point",
    button_style="primary",
    disabled=False,
    tooltip="Click to add this point to the list.",
)
add_point.on_click(handle_add)

remove_point = Button(
    description="Remove Point(s)",
    button_style="danger",
    disabled=False,
    tooltip="Click to remove selected point(s) from the list.",
)
remove_point.on_click(handle_remove)

start_date = date_widget("Start Date:")
end_date = date_widget("End Date:")

In [7]:
m.add_layer(p)
m.add_layer(f)
m.add_layer(c)

m += layer_control

m.on_interaction(handle_click)
m.add_control(legend)

display(m)
display(start_date, end_date)
display(latitude, longitude)
display(add_point, point_list, remove_point, run)

LayerException: layer already on map: Polygon(color='blue', locations=[(57.46875, -121.84375), (57.46875, -121.78125), (57.46875, -121.71875), (57.46875, -121.65625), (57.46875, -121.59375), (57.46875, -121.53125), (57.46875, -121.46875), (57.46875, -121.40625), (57.46875, -121.34375), (57.53125, -121.28125), (57.46875, -121.28125), (57.53125, -121.21875), (57.46875, -121.21875), (57.53125, -121.15625), (57.46875, -121.15625), (57.46875, -121.09375), (57.46875, -121.03125), (57.40625, -121.03125), (57.40625, -120.96875), (57.40625, -120.90625), (57.40625, -120.84375), (57.40625, -120.78125), (57.40625, -120.71875), (57.46875, -120.65625), (57.40625, -120.65625), (57.46875, -120.59375), (57.34375, -120.65625), (57.40625, -120.59375), (57.40625, -120.53125), (57.46875, -120.46875), (57.40625, -120.46875), (57.46875, -120.40625), (57.40625, -120.40625), (57.40625, -120.34375), (57.34375, -120.40625), (57.34375, -120.34375), (57.28125, -120.40625), (57.28125, -120.34375), (57.21875, -120.4

In [9]:
def client():
    flask_app = create_app()

    # Create a test client using the Flask application configured for testing
    return flask_app.test_client()


def full_rvic_test(url, client, valid_input=True):
    # input_params = urlencode(url)
    input_url = f"/osprey/input?{url}"

    input_response = client.get(input_url)
    if valid_input:
        assert input_response.status_code == 202
    else:
        assert input_response.status_code == 400
        return

    status_url = input_response.data.split()[-1].decode("utf-8")
    status_response = client.get(status_url)

    timeout = 1800  # Time to timeout in seconds
    for i in range(timeout):
        if status_response.data != b"Process is still running.":  # Process is completed
            break
        time.sleep(1)
        status_response = client.get(status_url)
    assert b"Process completed." in status_response.data

    output_url = status_response.data.split()[-1].decode("utf-8")
    output_response = client.get(output_url)
    streamflow_path = output_response.headers.get("Location")
    assert requests.get(streamflow_path).status_code == 200

In [10]:
test_client = client()

In [43]:
def build_url(start, end, points):
   
    start_parsed = str(start.time()).split(":")

    start_val = "&run_startdate=" + str(start.date()) + "-" + start_parsed[0]
    end_val = "&stop_date=" + str(end.date())
    
    lon_val = "&lons=" + ",".join([point.split(", ")[1] for point in points])
    lat_val = "&lats=" + ",".join([point.split(", ")[0] for point in points])

    url = "case_id=sample" + start_val + end_val + lon_val + lat_val

    print(url)

    return url


url = build_url(start_date.value, end_date.value, points)
full_rvic_test(url, test_client, valid_input=True)

case_id=sample&run_startdate=2020-10-06-06&stop_date=2020-10-20&lons=-116.78836&lats=44.02021
 owslib.wps.WPSException : {'code': 'NoApplicableCode', 'locator': 'None', 'text': 'Process error: ValueError: Some of the times specified were not found in the nctime variable.'}


MissingSchema: Invalid URL 'None': No scheme supplied. Perhaps you meant http://None?

In [ ]:
#enter host
#enter port
